In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#pd.set_option('display.max_columns', None,'display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
Member = pd.read_csv('~/Box Sync/KenyaPovertyTargetingModel/1data/KSPS/UNHCR-KEN-2018-SEA-Kalobeyei-Member.csv')
Nonfood = pd.read_csv('~/Box Sync/KenyaPovertyTargetingModel/1data/KSPS/UNHCR-KEN-2018-SEA-Kalobeyei-Nonfood.csv')

In [3]:
# Nonfood cleaning

Nonfood = Nonfood.rename(columns={"s1q2":"hhlocation", 
                        "s12q9":"purchased", 
                        "s12q10": "cost"})

Nonfood = Nonfood.drop(['weight','g_recall','nfcons_value',
                        'nfcons_value_capita','popweight',
                        's12q11'], axis=1)

NFSum = pd.DataFrame(Nonfood.groupby('hhid')['cost'].sum())

In [4]:
# Member cleaning
Member = Member.rename(columns={"s1q2":"hhlocation",
                                "s1q5":"interviewConsent",
                                "s2q8":"mainRespondent",
                                "s2q18":"hhHead",
                            #Demographic
                                "s2q11":"gender",
                                "s2q12":"ageRange",
                                "s2q13":"countryOfOrigin",
                            #Education
                                "s4q1":"anyEducation",
                                "s4q2":"currentlyInSchool",
                                "s4q3":"currentGrade",
                                "s4q4":"highestEducation",
                            ## Education -- Skills
                                "s4q5":"speaks",
                                "s4q6":"reads",
                                "s4q7":"writes",
                                "s4q8":"usesTech",
                                "s4q9":"drives",
                            # Employment 
                                "s5q1":"employee",
                                "s5q2":"employer",
                                "s5q3":"farmWorker",
                                "s5q4":"farmHelper",
                                "s5q5":"apprentice",
                                "s5q6":"volunteer",
                                "s5q20":"occupationStatus",
                                "s5q24":"numberOfIncomeActivities7days",
                            # Disabilities
                                "s9q2":"DifficultSeeing",
                                "s9q2_a":"glasses",
                                "s9q3":"DifficultHearing",
                                "s9q3_a":"hearingAid",
                                "s9q4":"difficultWalking",
                                "s9q5":"difficultMemory",
                                "s9q6":"difficultSelfCare",
                                "s9q7":"difficultCommunicate"})


Member = Member.drop(['weight',
# s5q7-14 ask about absenteeism at work
's5q7','s5q8','s5q9','s5q10','s5q11','s5q12','s5q13_1','s5q13_2','s5q13_3','s5q14','s5q14_b',
# s5q17-19 asks about what steps were taken in subject's job search
's5q17','s5q18','s5q19',
# s5q22 "How soon would he/she be able to start work if offered one"                 
's5q22',
# s5q23 is covered in s5q20
's5q23',
# Dr. told me to remove these
's5q25','s5q15','s5q16','s5q21','mainRespondent',
# Mean was 1                    
'interviewConsent' ,  
# Included in other sets
'ind_unique','indid','hhlocation',"s2q7a"
], axis=1)

MemberRaw=Member.copy()


In [22]:
## Cleaning variable inputs
Member.hhHead = MemberRaw.hhHead.map(dict(Yes=1, No=0))
Member.gender = MemberRaw.gender.map(dict(Female=1, Male=0))
Member.anyEducation = MemberRaw.anyEducation.map(dict(Yes=1, No=0))




In [35]:
## Age Manipulation
dfAge=Member[['hhid','ageRange','gender','currentlyInSchool','anyEducation']].copy()

ageDict = {"00 to 04": 0,
           "05 to 09": 1,
           "10 to 14": 2,
           "15 to 19": 3,
           "20 to 24": 4,
           "25 to 29": 5,
           "30 to 44": 6,
           "35 to 39": 7,
           "40 to 44": 8,
           "45 to 49": 9,
           "50 to 54": 10,
           "55 to 59": 11,
           "60 or more": 12}

dfAge.ageRange=dfAge.ageRange.replace(ageDict)

# Find cuttoff age for testing whether they have completed school
inSchoolDF = pd.DataFrame({'4-':dfAge.query('ageRange==0&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '5-9':dfAge.query('ageRange==1&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '10-14':dfAge.query('ageRange==2&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '15-19':dfAge.query('ageRange==3&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '20-24':dfAge.query('ageRange==4&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '25-29':dfAge.query('ageRange==5&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '30-34':dfAge.query('ageRange==6&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '35-39':dfAge.query('ageRange==7&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '40-44':dfAge.query('ageRange==8&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '45-49':dfAge.query('ageRange==9&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '50-54':dfAge.query('ageRange==10&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '55-59':dfAge.query('ageRange==11&anyEducation==1').currentlyInSchool.value_counts(dropna=False),
                '60+'  :dfAge.query('ageRange==12&anyEducation==1').currentlyInSchool.value_counts(dropna=False)
             }).fillna(0)

#find the age at which we can say that
#given a subject has an education & are not currently in school, 
# we can be 95% certain they are under this age 

inSchoolDF.sum(axis=1)
# No=675

inSchoolDF

# https://docs.google.com/spreadsheets/d/18tAtCwhdQKOA3tzjFUZ9xm5oZi-YzRmH1GZioLAcs-Y/edit#gid=1847526186

,4-,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60+
No,4,21,17,67,156,145,133,70,34,16,7,0.0,5
Yes,194,1208,1063,704,203,77,46,20,16,5,5,0.0,1


In [ ]:
# Create DF to test data regarding head of household
HeadMember = Member.query('hhHead==1')


pd.DataFrame({
# Column to count hhsize
    'size': Member.groupby('hhid')['gender'].count(),
# Colomn for houshold head is female 
    'headFemale':HeadMember.groupby('hhid')['gender'].mean(),
# Column for share of house is female
    'shareFemale':Member.groupby('hhid')['gender'].mean(),
# Share by Age stuff
    'shareAgeZeroToFour':dfAge.query('ageRange==0').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeFiveToNine':dfAge.query('ageRange==1').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeTenToFourteen':dfAge.query('ageRange==2').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeFifteenToTwentyfour':dfAge.query('ageRange==3|ageRange==4').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeTwentyfivetofourtynine':dfAge.query('ageRange==5|ageRange==6|ageRange==7|ageRange==8|ageRange==9').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeFiftyPlus':dfAge.query('ageRange==10|ageRange==11|ageRange==12').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareMaleAgeTenToNineteen':dfAge.query('gender==0&(ageRange==2|ageRange==3)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareFemaleAgeTenToNineteen':dfAge.query('gender==1&(ageRange==2|ageRange==3)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareMaleAgeTwentytoFourtynine':dfAge.query('gender==0&(ageRange==4|ageRange==5|ageRange==6|ageRange==7|ageRange==8|ageRange==9)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareFemaleAgeTwentytoFourtynine':dfAge.query('gender==1&(ageRange==4|ageRange==5|ageRange==6|ageRange==7|ageRange==8|ageRange==9)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count()
    }).fillna(0)



## TODO:
## Share for each country of origin
## Share over 20 and highest/current level of education
## Share speaks
## Share reads
## Share Writes



In [ ]:
type_counts = Member['currentGrade'].value_counts()